# Integration Tests
This notebook demonstrates a simple end-to-end run using the registry-based agent factory.
Each test is preceded by a markdown cell describing the goal and expected output.

## Setup
Patch the OpenAI engine and tool provider so the notebook can run without external dependencies.

In [ ]:
import os, sys, types
from pathlib import Path

# Fake engine avoids network calls
class FakeEngine:
    def call_engine(self, prompt: str, config: dict):
        return 'fake response'

# Stub jinja2 Template to a pass-through string
sys.modules['jinja2'] = types.SimpleNamespace(Template=lambda x: x)

# Provide a minimal tool provider that just echoes the input
from app.utils.tools.tool_result import ToolResult
class DummyToolProvider:
    def run_tool(self, name: str, code: str) -> ToolResult:
        return ToolResult(tool=name, status='success', output=code, metadata={})

fake_engine_module = types.SimpleNamespace(OpenAiGpt4oAgentEngineRunner=FakeEngine)
sys.modules['app.utils.engine.openai_gpt4o_runner'] = fake_engine_module
sys.modules['app.tool_providers.preprocessing_tool_provider'] = types.SimpleNamespace(PreprocessingToolProvider=DummyToolProvider)
os.environ['OPENAI_API_KEY'] = 'fake'


## Test 1: Build a generator agent
We build an agent using `build_agent`. The result should be an instance of `GeneratorAgent`.

In [ ]:
from app.utils.agent_factory import build_agent
from app.registries.agent_registry import AGENT_REGISTRY
from app.utils.symbol_graph.symbol_service import SymbolService

symbol_service = SymbolService(Path('tests/data/test_context_provider_preprocessor'))
agent = build_agent('generator', symbol_service)
type(agent).__name__

## Test 2: Run the agent
Running the agent on a test file should return a dictionary containing the generated response.

In [ ]:
log_path = Path('integration_logs/prompt_log.jsonl')
log_path.parent.mkdir(exist_ok=True)

result = agent.run({
    'symbol': 'my_symbol',
    'file': 'tests/data/test_context_provider_preprocessor/example.py',
    'experiment_id': 'demo',
    'run_id': '1',
    'round': 1,
    'system': 'demo',
    'log_path': log_path
})
result['response'][:20] + '...'

## Test 3: Inspect the log output
Read the JSONL log file and display its contents to verify that logging occurred.

In [ ]:
import json
log_entries = [json.loads(line) for line in log_path.read_text().splitlines()]
log_entries